# 1. Introduction

<div style="color:white;display:fill;
            background-color:#e38e05;font-size:150%;
            font-family:Nexa;letter-spacing:0.5px">
    <p style="padding: 4px;color:white;"><b>1.1 Background</b></p>
</div>

Welcome to this competition on predicting **Cervical Spine Fractures** from **Computed Tomography (CT) scans**. The **motivation** to use AI for this task is that a **quick** diagnosis can **reduce** the chance of **neurologic deterioration** and paralysis after trauma.

<center>
<img src="https://www.holisticbodyworks.com.au/wp-content/uploads/2018/05/Thoracic-Spine.jpg" width=400>
</center>

**Dataset origin**

The dataset we are using is made up of roughly **3000 CT studies**, [from twelve locations and across six continents](https://www.kaggle.com/competitions/rsna-2022-cervical-spine-fracture-detection/overview/acknowledgements). Spine **radiology specialists** have provided **annotations** to indicate the presence, vertebral level and location of any cervical spine fractures.

Special thanks to the competition **hosts** for providing such a comprehensive dataset:
* *Radiological Society of North America (RSNA)*
* *American Society of Neuroradiology (ASNR)*
* *American Society of Spine Radiology (ASSR)*

<div style="color:white;display:fill;
            background-color:#e38e05;font-size:150%;
            font-family:Nexa;letter-spacing:0.5px">
    <p style="padding: 4px;color:white;"><b>1.2 Evaluation metric</b></p>
</div>

We need to predict the **probability of fracture** for each of the **seven cervical vertebrae** denoted by C1, C2, C3, C4, C5, C6 and C7 as well as an **overall probability** of any fractures in the cervical spine. This means there will be **8 rows per image id**. Note that fractures in the skull base, thoracic spine, ribs, and clavicles are **ignored**.
    
The **competition metric** is a **weighted multi-label logarithmic loss** (average across all rows)
    
$$
L_{ij} = - w_j \left(y_{ij} \log(p_{ij}) + (1-y_{ij}) \log(1-p_{ij})  \right)
$$

where the **weights** [are given by](https://www.kaggle.com/competitions/rsna-2022-cervical-spine-fracture-detection/discussion/340392)
    
$$
w_{j} = \begin{cases}
1, \qquad \text{if vertebrae negative} \\
2, \qquad \text{if vertebrae positive} \\
7, \qquad \text{if patient negative} \\
14, \qquad \text{if patient negative}
\end{cases}
$$

<br>
<center>
<img src="https://i.postimg.cc/wBYCYqFG/metric-plot.png" width=600>
</center>
<br>

Notice how **more weight** is put on **positive cases** and and the **most weight** on the **overall probability** of any fractures.

<hr>

**Example:**

Below is a table going through an example of **how the evaluation metric is calculated** for a single case id. This would then be **averaged** across all row id's.

<center>
<img src="https://i.postimg.cc/nh2zCP5T/61415.jpg" width=750>
</center>

Note: it is convention to use the **natural logarithm** (base e) for calculating the log loss. 

<div style="color:white;display:fill;
            background-color:#e38e05;font-size:150%;
            font-family:Nexa;letter-spacing:0.5px">
    <p style="padding: 4px;color:white;"><b>1.3 Code requirements</b></p>
</div>

This is a **code competition**, which means that **submissions** are made through **notebooks**. Furthermore, the submission notebook is subject to these conditions:

* **run-time** (CPU/GPU) **<= 9 hours**
* **internet** access **disabled**
* **external data is allowed**, including pre-trained models
* the submission file must be named **submission.csv**
              
Note that the **test set is hidden**, and will populated when you submit your notebook. 

<hr>

*Competition timeline:*
    
* Start date - 28th July 2022
* Finish date - 27th October 2022

<div style="color:white;display:fill;
            background-color:#e38e05;font-size:150%;
            font-family:Nexa;letter-spacing:0.5px">
    <p style="padding: 4px;color:white;"><b>1.4 Libraries</b></p>
</div>

In [ ]:
! pip install pylibjpeg pylibjpeg-libjpeg pydicom
! pip install -U python-gdcm

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import seaborn as sns
import cv2
import os
from os import listdir
import re
import gc
import gdcm
import pydicom
from pydicom import dcmread
import pylibjpeg
from pydicom.pixel_data_handlers.util import apply_voi_lut
import scipy.ndimage
from tqdm import tqdm
from pprint import pprint
from time import time
import itertools
from skimage import measure 
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import nibabel as nib
from glob import glob
import warnings
import dask.array as da
%matplotlib inline
sns.set(style='darkgrid', font_scale=1.6)

# 2. Data

<div style="color:white;display:fill;
            background-color:#e38e05;font-size:150%;
            font-family:Nexa;letter-spacing:0.5px">
    <p style="padding: 4px;color:white;"><b>2.1 Data frames</b></p>
</div>


In [ ]:
# Load metadata
train_df = pd.read_csv("../input/rsna-2022-cervical-spine-fracture-detection/train.csv")
train_bbox = pd.read_csv("../input/rsna-2022-cervical-spine-fracture-detection/train_bounding_boxes.csv")
test_df = pd.read_csv("../input/rsna-2022-cervical-spine-fracture-detection/test.csv")
ss = pd.read_csv("../input/rsna-2022-cervical-spine-fracture-detection/sample_submission.csv")

# Print dataframe shapes
print('train shape:', train_df.shape)
print('train bbox shape:', train_bbox.shape)
print('test shape:', test_df.shape)
print('sample shape:', ss.shape)
print('')

# Show first few entries
train_df.head()

> **train.csv** - contains metadata for train_images.
> * StudyInstanceUID - The study ID. There is one unique study ID for each patient scan.
> * patient_overall - The patient level outcome, i.e. if any of the vertebrae are fractured.
> * C[1-7] - Whether the given vertebrae is fractured.

In [ ]:
train_bbox.head()

> **train_bounding_boxes.csv** - contains bounding boxes of where fractures occured for a subset of the training set.
> * StudyInstanceUID - The study ID. There is one unique study ID for each patient scan.
> * x - x-coordinate of bounding box bottom left corner
> * y - y-coordinate of bounding box bottom left corner
> * width - width of bounding box
> * height - height of bounding box
> * slice_number - slice number of scan

Note: we only have bounding boxes for a [subset](https://www.kaggle.com/competitions/rsna-2022-cervical-spine-fracture-detection/discussion/343105) of the training set. We'll explore the exact proportion later on.

In [ ]:
test_df.head()

> **test.csv** - contains metadata for test_images.
> * row_id - The row ID. This will match the same column in the sample submission file.
> * StudyInstanceUID - The study ID.
> * prediction_type - Which one of the eight target columns needs a prediction in this row.

Note: The full test set will be **populated at inference time**.

In [ ]:
ss.head()

> **sample_submission.csv** - A valid sample submission.
> * row_id - The row ID. See the test.csv for what prediction needs to be filed in that row.
> * fracture - The target column.

# 3. EDA - METADATA
<div style="color:white;display:fill;
            background-color:#e38e05;font-size:150%;
            font-family:Nexa;letter-spacing:0.5px">
    <p style="padding: 4px;color:white;"><b>3.1 Fracture distributions</b></p>
</div>

* The overall target is roughly **balanced** (52/48 split). 
* **C7** has the **highest proportion of fractures** (19%) whereas **C3** has the **lowest** (4%) which is concistent with https://www.sciencedirect.com/topics/medicine-and-dentistry/third-cervical-vertebra. 
* Several patients have **more than one** fracture.
* If **multiple fractures** occur on a single patient, they tend to occur in vertebrae **close together**, e.g. C4 & C5 as opposed to C1 & C7.

In [ ]:
plt.figure(figsize=(20,6))
plt.subplot(1,2,1)
ax1 = sns.countplot(data=train_df, x='patient_overall')
for container in ax1.containers:
    ax1.bar_label(container)
plt.title('Fractures by patient')
plt.ylim([0,1300])

# Unpivot train_df for plotting
train_melt = pd.melt(train_df, id_vars = ['StudyInstanceUID', 'patient_overall'],
             value_vars = ['C1','C2','C3','C4','C5','C6','C7'],
             var_name="Vertebrae",
             value_name="Fractured")

plt.subplot(1,2,2)
ax2 = sns.countplot(data=train_melt, x='Vertebrae', hue='Fractured')
for container in ax2.containers:
    ax2.bar_label(container)
plt.title('Fractures by vertebrae')
plt.ylim([0,2800])

In [ ]:
plt.figure(figsize=(12,5))
ax = sns.countplot(x = train_df[['C1','C2','C3','C4','C5','C6','C7']].sum(axis=1))
for container in ax.containers:
    ax.bar_label(container)
plt.title('Number of fractures by patient')
plt.xlabel('Number of fractures')
plt.ylim([0,1300])

In [ ]:
# Heatmap of correlations
plt.figure(figsize=(6,5))
sns.heatmap(train_df[['C1','C2','C3','C4','C5','C6','C7']].corr(), cmap='bwr', vmin=-1, vmax=1)
plt.title('Correlations')

<div style="color:white;display:fill;
            background-color:#e38e05;font-size:150%;
            font-family:Nexa;letter-spacing:0.5px">
    <p style="padding: 4px;color:white;"><b>3.2 Study Id's</b></p>
</div>

The cases in the dataset have **unique id's** like '1.2.826.0.1.3680043.6200'. It turns out only the **number after the last full stop** is important. 

In [ ]:
# Example
train_df['StudyInstanceUID'][0]

In [ ]:
# Find unique numbers in study id's
for i in range(7):
    print(train_df['StudyInstanceUID'].map(lambda x : x.split('.')[i]).unique())

# 4. EDA - Train images

<div style="color:white;display:fill;
            background-color:#e38e05;font-size:150%;
            font-family:Nexa;letter-spacing:0.5px">
    <p style="padding: 4px;color:white;"><b>4.1 What is DICOM?</b></p>
</div>

A **.dcm** file follows the **Digital Imaging and Communications in Medicine** (DICOM) format. It is the standard format used for storing **medical images** and **related metadata**. It dates back to 1983, although it has been revised many times. 

This **pixel size/coarseness** of the scan **differs** from scan to scan (e.g. the distance between slices may differ), which can **hurt performance** of CNN approaches. We can deal with this by **isomorphic resampling**.

We can use the [pydicom library](https://pydicom.github.io/) to open and explore these files.

In [ ]:
ex_path = "../input/rsna-2022-cervical-spine-fracture-detection/train_images/1.2.826.0.1.3680043.12281/110.dcm"
dcm_example = pydicom.dcmread(ex_path)
dcm_example

>The **image data** is stored in an array under **'Pixel Data'**. Everything else is **metadata**.
>* The **'Rows'** and **'Columns'** values tell us the **image size**.
>* The **'Pixel Spacing'** and **'Slice Thickness'** tell us the **pixel size** and **thickness**. Together the form a **voxel**.
>* The **'Window Center'** and **'Window Width'** give information about the **brightness** and **contrast** of the image respectively.
>* The **'Rescale Intercept'** and **'Rescale Slope'** determine the range of pixel values. ([ref](https://stackoverflow.com/questions/10193971/rescale-slope-and-rescale-intercept)) and to tensform the pixel value into **HU(Hounsfield Unit)**.
>* **'ImagePositionPatient'** tells us the x, y, and z coordinates of the top left corner of each image in mm
>* **InstanceNumber** is the slice number.

The unit of measurement in CT scans is the **Hounsfield Unit (HU)**, which is a measure of radiodensity. CT scanners are carefully calibrated to accurately measure this.  From Wikipedia:

![HU examples][1]

By default however, the returned values are not in this unit. Let's fix this.

Some scanners have cylindrical scanning bounds, but the output image is square. The pixels that fall outside of these bounds get the fixed value -2000. The first step is setting these values to 0, which currently corresponds to air. Next, let's go back to HU units, by multiplying with the rescale slope and adding the intercept (which are conveniently stored in the metadata of the scans!).

  [1]: http://i.imgur.com/4rlyReh.png

In [ ]:
# Load the scans in given folder path
def load_scan(path):
    slices = [pydicom.read_file(path + '/' + s) for s in os.listdir(path)]
    slices.sort(key = lambda x: float(x.ImagePositionPatient[2])) # Sort the images based on the Z axis.
    return slices

In [ ]:
path="../input/rsna-2022-cervical-spine-fracture-detection/train_images/1.2.826.0.1.3680043.12281" #train_df.loc[0,'StudyInstanceUID']
patient=load_scan(path)

>### Adding further information

>1. The CT-scan captures information about the radiodensity of an object or tissue exposed to x-rays. A transversal slice of a scan is reconstructed after taking measurements from several different directions.
>2. We need to transform to Hounsfield units as the spectral composition of the x-rays depends on the measurement settings like acquisition parameters and tube voltage. By normalizing to values of water and air (water has HU 0 and air -1000) the images of different measurements are becoming comparable.
>3. A ct-scanner yields roughly 4000 grey values that can't be captured by our eyes. This is why windowing is performed. This way the image is displayed in a HU range that suites most to the region of interest. 

In [ ]:
def get_pixels_hu(slices):
    image = np.stack([cv2.resize(s.pixel_array,(512,512),interpolation = cv2.INTER_NEAREST) for s in slices])
    
    # Convert to int16 (from sometimes int16), 
    # should be possible as values should always be low enough (<32k)
    image = image.astype(np.int16)
    image = da.from_array(image)
    # Set outside-of-scan pixels to 0
    # The intercept is usually -1024, so air is approximately 0
    
    
    image[image <= -1000] = 0
    
    # Convert to Hounsfield units (HU)
        
    intercept = da.from_array([slices[slice_number].RescaleIntercept for slice_number in range(len(slices))])
    slope = da.from_array([slices[slice_number].RescaleSlope for slice_number in range(len(slices))])
    
    intercept=intercept.reshape((-1,1,1))
    slope=slope.reshape((-1,1,1))
    
    image= slope * image.astype("float64")
    image= image.astype("int16")
        
    image+= intercept
     
    return image.astype("int16")

In [ ]:
patient_slice=get_pixels_hu(patient)

In [ ]:
#Ploting pixel array
plt.imshow(patient[110].pixel_array,cmap='bone')
plt.axis("off")

In [ ]:
#Ploting pixel array distribution
plt.hist(patient[110].pixel_array.flatten(),color="r",bins=50)
plt.xlabel("Pixel Values")
plt.ylabel("Fequency")
plt.show()

In [ ]:
#Ploting HU array
plt.imshow(patient_slice[110],cmap='bone')
plt.axis("off")

In [ ]:
#Ploting HU distribution
plt.hist(patient_slice[110].flatten().compute(),color="r",bins=50)
plt.xlabel("HU Values")
plt.ylabel("Fequency")
plt.show()

<br>
<center>
    <h1>Reference <b>Hounsfield Unit (HU)</b> values for vertebral column</h1>
</center>
<br>

<br>
<center>
<img src="https://www.frontiersin.org/files/Articles/920167/fendo-13-920167-HTML/image_m/fendo-13-920167-t002.jpg" width=800>
</center>
<br>

<div style="color:white;display:fill;
            background-color:#e38e05;font-size:150%;
            font-family:Nexa;letter-spacing:0.5px">
    <p style="padding: 4px;color:white;"><b>4.3 Normalization</b></p>
</div>

As HU values range from **100** to **3000**. So we will use this values for normalization.

In [ ]:
MIN_BOUND = 100
MAX_BOUND = 3000.0
    
def normalize(image):
    image = (image - MIN_BOUND) / (MAX_BOUND - MIN_BOUND)
    image[image>1] = 1.
    image[image<0] = 0.
    return image

In [ ]:
image=normalize(patient_slice)

In [ ]:
plt.imshow(image[110],cmap="bone")
plt.axis("off")

<div style="color:white;display:fill;
            background-color:#e38e05;font-size:150%;
            font-family:Nexa;letter-spacing:0.5px">
    <p style="padding: 4px;color:white;"><b>4.4 Zero centering</b></p>
</div>

As a final preprocessing step, it is advisory to **zero center** your data so that your **mean value is 0**. To do this you simply **subtract** the mean pixel value from all pixels.

In [ ]:
PIXEL_MEAN = 0.016076984448095525

def zero_center(image):
    image = image - PIXEL_MEAN
    return image

In [ ]:
image=zero_center(image)

In [ ]:
plt.imshow(image[110],cmap="bone")
plt.axis("off")

In [ ]:
#Delete all unused objects to free up memory
del ax1
del ax2
del ax
del ex_path
del dcm_example
del path
del patient
del patient_slice
del image

gc.collect()

In [ ]:
#To preprocess all the patient folder and download it
import zipfile
import bz2
import _pickle as cPickle

def save_image():
    pixel_sum=0
    pixel_number=0
    with zipfile.ZipFile('final.zip', 'w') as zipF:
        for patient_ID in tqdm(train_df['StudyInstanceUID']):
            path="../input/rsna-2022-cervical-spine-fracture-detection/train_images/"+patient_ID
            patient_pixel=load_scan(path)
            patient_hu=get_pixels_hu(patient_pixel)
            patient_hu_normalised=normalize(patient_hu)
            patient_hu_normalised_centered=zero_center(patient_hu_normalised)
            
            with bz2.BZ2File(patient_ID+'.pbz2', 'w') as f: 
                cPickle.dump(patient_hu_normalised_centered, f)
            zipF.write(patient_ID+'.pbz2', compress_type=zipfile.ZIP_DEFLATED, compresslevel=9)
            os.remove(patient_ID+'.pbz2')
            
            del path
            del patient_pixel
            del patient_hu
            del patient_hu_normalised
            del patient_hu_normalised_centered
            gc.collect()

In [ ]:
#Uncomment to save the images
#We cannot preprocess the image here as it would take a lot of time to save it so we will do the preprocessing where we will build the model
#save_image()

This is great, but at the moment we don't know which vertebrae is being shown in each image. One way to work this out is by using the **segmentations**.

# 5. Segmentations

<div style="color:white;display:fill;
            background-color:#e38e05;font-size:150%;
            font-family:Nexa;letter-spacing:0.5px">
    <p style="padding: 4px;color:white;"><b>5.1 What is NIfTI? </b></p>
</div>

A **.nii** file follows the **Neuroimaging Informatics Technology Initiative** (NIfTI) format. Compared to the DICOM, NIfTI is **simpler** and **easier** to support. 

To open .nii files we can use the [nibabel library](https://nipy.org/nibabel/gettingstarted.html).

In [ ]:
def load_NIfTI(path):
    nii_example = nib.load(path)

    # Convert to numpy array
    seg = nii_example.get_fdata()
    print(seg.shape)
    
    return seg

In [ ]:
path="../input/rsna-2022-cervical-spine-fracture-detection/segmentations/1.2.826.0.1.3680043.12281.nii" #train_df.loc[0,'StudyInstanceUID']
segment_image=load_NIfTI(path)

Each nifti file contains segmentations for **all slices** in a scan. However, we need to be careful about the **orientation** of the segmentations.

> Please be aware that the NIFTI files consist of segmentation in the sagittal plane, while the DICOM files are in the axial plane.

The correct way to deal with this is explained in this [discussion post](https://www.kaggle.com/competitions/rsna-2022-cervical-spine-fracture-detection/discussion/340612). We need to use the following line:

In [ ]:
# Align orientation with images
segment_image = segment_image[:, ::-1, ::-1].transpose(2, 1, 0)
segment_image.shape


<div style="color:white;display:fill;
            background-color:#e38e05;font-size:150%;
            font-family:Nexa;letter-spacing:0.5px">
    <p style="padding: 4px;color:white;"><b>5.2 Exploring the masks</b></p>
</div>

In [ ]:
# Plot images
fig, axes = plt.subplots(nrows=3, ncols=6, figsize=(24,12))
fig.suptitle(f'ID: 1.2.826.0.1.3680043.12281', weight="bold", size=20)

for i in range(110,128):
    mask = segment_image[i]
    slice_no = i

    # Plot the image
    x = (i-110) // 6
    y = (i-110) % 6

    axes[x, y].imshow(mask, cmap='inferno')
    axes[x, y].set_title(f"Slice: {slice_no}", fontsize=14, weight='bold')
    axes[x, y].axis('off')

Compare these with the previous images. These masks give us the **location** of the vertebrae, which is very helpful because we know the fractures can only occur in these regions.

They also tells us which **vertebrae** are in the images. By looking at the unique values in each slice, we find a 0 for the background and another number like 3 for vertebrae C3. 

In [ ]:
np.unique(segment_image[110])

Unfortunately, we don't have segmentations for the whole train set. This means we will have to extract the **vertebrae numbers** from the **.dcm files**. An idea to keep in mind is that maybe we could train a segmentation model to predict the segmentation masks for the rest of the train and all of the test images.

# 6. Extract metadata

<div style="color:white;display:fill;
            background-color:#e38e05;font-size:150%;
            font-family:Nexa;letter-spacing:0.5px">
    <p style="padding: 4px;color:white;"><b>6.1 Save metadata </b></p>
</div>

This section uses code from Andrada Olteanu.

In [ ]:
# From https://www.kaggle.com/code/andradaolteanu/rsna-fracture-detection-dicom-images-explore
def get_observation_data(path):
    '''
    Get information from the .dcm files
    '''

    dataset = pydicom.read_file(path)
    
    # Dictionary to store the information from the image
    observation_data = {
        "Rows" : dataset.get("Rows"),
        "Columns" : dataset.get("Columns"),
        "SOPInstanceUID" : dataset.get("SOPInstanceUID"),
        "ContentDate" : dataset.get("ContentDate"),
        "SliceThickness" : dataset.get("SliceThickness"),
        "InstanceNumber" : dataset.get("InstanceNumber"),
        "ImagePositionPatient" : dataset.get("ImagePositionPatient"),
        "ImageOrientationPatient" : dataset.get("ImageOrientationPatient"),
    }

    # String columns
    str_columns = ["SOPInstanceUID", "ContentDate", 
                   "SliceThickness", "InstanceNumber"]
    for k in str_columns:
        observation_data[k] = str(dataset.get(k)) if k in dataset else None

    return observation_data

def get_metadata():
    '''
    Retrieves the desired metadata from the .dcm files and saves it into dataframe.
    '''
    
    exceptions = 0
    dicts = []

    for k in tqdm(range(len(train_df))):
        if (k % 100)==0:
            print(f'Iteration: {k}')
            
        dt = train_df.iloc[k, :]

        # Get all .dcm paths for this Instance
        dcm_paths = glob(f"{base_path}/train_images/{dt.StudyInstanceUID}/*")

        for path in dcm_paths:
            try:
                # Get datasets
                dataset = get_observation_data(path)
                dicts.append(dataset)
            except Exception as e:
                exceptions += 1
                continue

    # Convert into df
    meta_train_data = pd.DataFrame(data=dicts, columns=md_example.keys())
    
    # Export information
    meta_train_data.to_csv("meta_train.csv", index=False)
    
    print(f"Metadata created. Number of total fails: {exceptions}.")

In [ ]:
# Example
ex_path="../input/rsna-2022-cervical-spine-fracture-detection/train_images/1.2.826.0.1.3680043.12281/110.dcm"
md_example = get_observation_data(ex_path)
pprint(md_example)

In [ ]:
# Create and save the metadata (~ 2 hours)
#get_metadata()

<div style="color:white;display:fill;
            background-color:#e38e05;font-size:150%;
            font-family:Nexa;letter-spacing:0.5px">
    <p style="padding: 4px;color:white;"><b>6.2 Explore metadata </b></p>
</div>

In [ ]:
# Read in saved metadata
meta_train = pd.read_csv("../input/rsna-2022-spine-fracture-detection-metadata/meta_train.csv")
meta_train["StudyInstanceUID"] = meta_train["SOPInstanceUID"].apply(lambda x: ".".join(x.split(".")[:-2]))
print('meta_train shape:', meta_train.shape)
meta_train.head(3)

In [ ]:
# Image size
meta_train["ImageSize"] = meta_train["Rows"].astype(str) + " x " + meta_train["Columns"].astype(str)

# Plot image sizes
plt.figure(figsize=(18, 5))
ax = sns.countplot(data=meta_train, x="ImageSize")
for container in ax.containers:
    ax.bar_label(container)
plt.ylim([0,800000])
plt.title('Image sizes in train images', fontsize=25, y=1.02)

Almost all images have size 512x512. We should therefore resize the other images to size 512x512.

**Content Date**

In [ ]:
# Unique values
meta_train['ContentDate'].unique()

We can drop this feature as it is constant.

In [ ]:
meta_train.drop('ContentDate', axis=1, inplace=True)

**Slice Thickness**

In [ ]:
meta_train["SliceThickness"].unique()

In [ ]:
# Plot slice thickness
plt.figure(figsize=(18, 5))
ax = sns.countplot(data=meta_train, x="SliceThickness")
for container in ax.containers:
    ax.bar_label(container)
ax.set_xticklabels(['0.488','0.500','0.600','0.60...2','0.625','0.664','0.670','0.75','0.800','0.900','1.000'])
plt.ylim([0,420000])
plt.title('Slice thickness distribution', fontsize=25, y=1.02)

**Number of slices per scan**

In [ ]:
# Slice counts
slice_counts = meta_train["StudyInstanceUID"].value_counts().reset_index()
slice_counts.columns = ["StudyInstanceUID", "count"]

# Distribution of slices counts
plt.figure(figsize=(18, 5))
sns.histplot(data=slice_counts, x="count", kde=True, bins=40)
plt.title("Number of slices by scan", size=25, y=1.02)
plt.xlabel("Number of Slices", size = 18)

**Image Position Patient**

In [ ]:
# Extract x, y, z coordinates of position vector
meta_train['ImagePositionPatient_x'] = meta_train['ImagePositionPatient'].apply(lambda x: float(x.replace(',','').replace(']','').replace('[','').split()[0]))
meta_train['ImagePositionPatient_y'] = meta_train['ImagePositionPatient'].apply(lambda x: float(x.replace(',','').replace(']','').replace('[','').split()[1]))
meta_train['ImagePositionPatient_z'] = meta_train['ImagePositionPatient'].apply(lambda x: float(x.replace(',','').replace(']','').replace('[','').split()[2]))

In [ ]:
# Plot position coordinates
plt.figure(figsize=(20,5))
plt.subplot(1,3,1)
sns.histplot(meta_train['ImagePositionPatient_x'])
plt.ylim([0,20000])
plt.title('x-coordinate', fontsize=25, y=1.02)

plt.subplot(1,3,2)
sns.histplot(meta_train['ImagePositionPatient_y'], color='C1')
plt.ylabel('')
plt.ylim([0,20000])
plt.title('y-coordinate', fontsize=25, y=1.02)

plt.subplot(1,3,3)
sns.histplot(meta_train['ImagePositionPatient_z'], color='C2')
plt.ylabel('')
plt.ylim([0,20000])
plt.title('z-coordinate', fontsize=25, y=1.02)

**Image Orientation Patient**

This feature tells us coordinates for the position of the patient in the scanner. It could tell us if the images are slightly rotated for example.

In [ ]:
meta_train['ImageOrientationPatient'].nunique()

In [ ]:
meta_train['ImageOrientationPatient'].unique()[:5]

In [ ]:
# Clean metadata
meta_train_clean = meta_train.drop(['SOPInstanceUID','ImagePositionPatient','ImageOrientationPatient','ImageSize'], axis=1)
meta_train_clean.rename(columns={"Rows": "ImageHeight", "Columns": "ImageWidth","InstanceNumber": "Slice"}, inplace=True)
meta_train_clean = meta_train_clean[['StudyInstanceUID','Slice','ImageHeight','ImageWidth','SliceThickness','ImagePositionPatient_x','ImagePositionPatient_y','ImagePositionPatient_z']]
meta_train_clean.sort_values(by=['StudyInstanceUID','Slice'], inplace=True)
meta_train_clean.reset_index(drop=True, inplace=True)

# Export information
#meta_train_clean.to_csv("meta_train_clean.csv", index=False)

# Preview first few columns
meta_train_clean.head(3)

The **cleaned version** of this metadata is also in SAMUEL CORTINHAS dataset [here](https://www.kaggle.com/datasets/samuelcortinhas/rsna-2022-spine-fracture-detection-metadata).

# 7. Bounding boxes

<div style="color:white;display:fill;
            background-color:#e38e05;font-size:150%;
            font-family:Nexa;letter-spacing:0.5px">
    <p style="padding: 4px;color:white;"><b> 7.1 Box distributions </b></p>
</div>

We are only given bounding boxes for a **subset** of the data. In particular, only **12%** of patients in the train set have any bounding box measurements.

This information is useful in telling us exactly where the fractures have occured. We could consider training an **object localisation** algorithm to provide bounding boxes for the whole train set. 

In [ ]:
print(f'Patients with bounding box measurements: {train_bbox["StudyInstanceUID"].nunique()} ({np.round(100*train_bbox["StudyInstanceUID"].nunique()/len(train_df),1)} %)' )

In [ ]:
plt.figure(figsize=(16,5))
sns.histplot(train_bbox["StudyInstanceUID"].value_counts().values, kde=True, bins=40)
plt.title('Number of slices with bounding boxes per patient')
plt.xlabel('Number of bboxes')

In [ ]:
# Plot images
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(24,12))

slice1 = 119
bbox_id1 = '1.2.826.0.1.3680043.25651'
file1 = pydicom.dcmread(f"../input/rsna-2022-cervical-spine-fracture-detection/train_images/{bbox_id1}/{slice1}.dcm")
img1 = apply_voi_lut(file1.pixel_array, file1)
info1 = train_bbox[(train_bbox['StudyInstanceUID']==bbox_id1)&(train_bbox['slice_number']==slice1)]
rect1 = patches.Rectangle((float(info1.x), float(info1.y)), float(info1.width), float(info1.height), linewidth=3, edgecolor='r', facecolor='none')

axes[0].imshow(img1, cmap="bone")
axes[0].add_patch(rect1)
axes[0].set_title(f"ID:{bbox_id1}, Slice: {slice1}", fontsize=20, weight='bold',y=1.02)
axes[0].axis('off')

slice2 = 156
bbox_id2 = '1.2.826.0.1.3680043.23817'
file2 = pydicom.dcmread(f"../input/rsna-2022-cervical-spine-fracture-detection/train_images/{bbox_id2}/{slice2}.dcm")
img2 = apply_voi_lut(file2.pixel_array, file2)
info2 = train_bbox[(train_bbox['StudyInstanceUID']==bbox_id2)&(train_bbox['slice_number']==slice2)]
rect2 = patches.Rectangle((float(info2.x), float(info2.y)), float(info2.width), float(info2.height), linewidth=3, edgecolor='r', facecolor='none')

axes[1].imshow(img2, cmap="bone")
axes[1].add_patch(rect2)
axes[1].set_title(f"ID:{bbox_id2}, Slice: {slice2}", fontsize=20, weight='bold',y=1.02)
axes[1].axis('off')